## Import the pipeline functions from functions.py

In [39]:
%run -i functions_edit.py

# **Users please enter the parameters of your experiment:**

In [3]:
# Default is channels=['CY5', 'CY3', 'CY3.5', 'DAPI']
channels = ['CY5', 'CY3', 'CY3.5', 'DAPI']

# Number of Z-slices per channel (for Marah's data this is 41)
number_zslices = 41

# Default is colors=['magenta', 'green', 'cyan', 'blue']
colors = ['magenta', 'green', 'cyan', 'blue']

# Defaults is voxel_size=(200, 64.5, 64.5)
voxel_size=(200, 64.5, 64.5)

# Default is spot_radius=(200, 70, 70)
spot_radius=(200, 70, 70)

# Please add the paths to the three Cellpose models downloaded from the repo.
pretrained_whole_model_path = "C:/Users/lotta/Pictures/Cellpose/whole_DAPI+DIC/models/whole_model_993"
# If you do not wish to use the Cellpose models below set their value to None as follows: 
# pretrained_separate_model_path = None
# pretrained_whole_SHIFTCORRECTED_model_path = None
pretrained_separate_model_path = "C:/Users/lotta/Pictures/Cellpose/sep_DAPI+DIC/models/sep_model_1180"
pretrained_whole_SHIFTCORRECTED_model_path = "C:/Users/lotta/Pictures/Cellpose/shifted_whole_DAPI+DIC/models/shifted_whole_model_993"


# Please add the path of the TIF and DIC file you'd like to analyse with this pipeline
tif_path = "C:/Users/lotta/Documents/Bioinformatics_Msc/Project/DATA_INITIAL/pair_3/yET915-SUN4Q570-SRL1CFR610-ASH1CLB2Q670_03_CY5, CY3.5 NAR, CY3, DAPI.tif"
dic_path = "C:/Users/lotta/Documents/Bioinformatics_Msc/Project/DATA_INITIAL/pair_3/yET915-SUN4Q570-SRL1CFR610-ASH1CLB2Q670_03_DIC-100.tif"

# Please add the path to which you'd like to save the Dataframe containing the metrics calculates
results_excel_path = 'C:/Users/lotta/Bioinformatics/smFISHimagepipeline/pipeline_outputs/output_yET916_30.xlsx'

#### Read corresponding DIC and TIF images and split the TIF file into the four channels
- "corresponding_files" is a list of the corresponding file names. 
- "stack_example" is an array of the 41 z-slices for each of the 4 channels.

### Load TIF and DIC image

In [4]:
corresponding_files, image_stack = read_stack([tif_path, dic_path], number_zslices, channels)

100%|████████████████████████████████████████████████████████████████████████████████| 164/164 [00:02<00:00, 73.68it/s]


#### For each channel create a Maximum Intensity Projection for each channel
- For each channel there are 40 slices of these not all are in focus, if the "in-focus-zslices" are known they can be input into the function. 
- If not a laplacian operator can be used to select these "in-focus-zslices".

In [5]:
# Create Maximum Intensity Projections for each channel
# Each MIP image array is iteratively added to the projection list.
projection = []
for i in image_stack:
    # Instead of choose_focus_lap(i) the focussed slices can be 
    # input in the form [first in focus slice, last in focus slice]
    infocus_slices = choosing_in_focus_slices(i, number_of_slices=12)
    projection.append(generating_maximum_projection(i, infocus_slices))

#### View the MIPs alongside the DIC in napari

In [6]:
# View the MIPs in napari.
viewer= napari.Viewer()
napari_view(corresponding_files, np.array(projection), channels, colors)

## Detect the mRNA spots for each of the zslices in each channel
- "spot_coord_dict" is a dictionary where the key is each channel and the value is an array of the mRNA coordinates.

In [7]:
# RNA detection for each channel.
spot_coord_dict = detecting_mRNA_spots(image_stack, voxel_size, spot_radius, channels)

In [42]:
spot_coord_dict

{'CY5': array([[   0, 1193,   11],
        [   0, 1434,  769],
        [   0, 1440,  766],
        ...,
        [  29, 2203, 1889],
        [  32, 1840, 1899],
        [  32, 2127, 1895]], dtype=int64),
 'CY3': array([[   0, 1327, 1153],
        [   9,  860, 1371],
        [  10,  919,  685],
        ...,
        [  33,  853, 1455],
        [  33, 1340, 1117],
        [  33, 1617, 2076]], dtype=int64),
 'CY3.5': array([[   9,  521, 2016],
        [  11,   34,  652],
        [  11,   58,  647],
        ...,
        [  30, 2139,  652],
        [  30, 2045, 1861],
        [  33, 1750,  412]], dtype=int64)}

#### View the MIPs alongside the DIC with the detected spots in napari

In [8]:
viewer= napari.Viewer()
napari_view(corresponding_files, np.array(projection), channels, colors)
napari_view_spots(corresponding_files, np.array(projection), spot_coord_dict, channels, colors)

# Cellpose
#### Load pre-trained models
- These can be substituted with any user-trained model and there is no need to use two, this is simply for mother-bud analysis. 

In [9]:
whole_model = models.CellposeModel(pretrained_model=pretrained_whole_model_path)
if pretrained_separate_model_path != None:
    sep_model = models.CellposeModel(pretrained_model=pretrained_separate_model_path)

#### Create a two channel image with the DAPI MIP and the DIC image to generate Cellpose masks

In [10]:
channel_dictionary = split_stack(image_stack, channels, colors)
in_focus_slices = choosing_in_focus_slices(channel_dictionary['DAPI'][1], number_of_slices=15)
dapi_projection = (generating_maximum_projection(channel_dictionary['DAPI'][1], in_focus_slices))
cellpose_input = [io.imread(corresponding_files[1]), dapi_projection]

#### Choose images to create masks of using the model

In [11]:
imgs = np.array(cellpose_input)
nimg = len(imgs)

#### Evaluate the images and return masks

In [12]:
# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
cellpose_channels = [[1,2]]
masks_whole, whole_flows, whole_styles = whole_model.eval(imgs, diameter=None, channels=cellpose_channels)
if pretrained_separate_model_path != None:
    masks_sep, sep_flows, sep_styles = sep_model.eval(imgs, diameter=None, channels=cellpose_channels)

In [13]:
projection_dict = create_mip_projection_dict(channels, projection)

### Assign mRNA spots to individual cells
##### For the 'separate' masks i.e. the mother and the bud are two separate units

In [14]:
# Load masks of both Cellpose models.
spots_per_cell_whole_masks = extracting_spots_per_cell(spot_coord_dict, masks_whole, projection_dict)

## Work out the DIC shift using the proportion of mRNA spots in cells vs outside of them 
- "dic_shift_coords" is very slow as it is currently enumerating over a vast number of coordinate combinations. Currently unsure how to solve this.

In [15]:
proportion_dict = {}

In [16]:
# Default is channels=['rna_coord', 'CY3', 'CY3.5']
spot_channels = list(spot_coord_dict.keys())
spot_channels[0] = 'rna_coord'

In [17]:
prop, cells, total = counting_spots_in_cells(spot_coord_dict, spots_per_cell_whole_masks, spot_channels)
print(f'Original proportion of mRNA spots per cell: {prop}')
dic_shift_info = finding_max_proportion_of_image(prop, 0, 0, proportion_dict, spot_coord_dict, masks_whole, projection_dict)

Original proportion of mRNA spots per cell: 68.56292117223006
Current maximum proportion 72.90 and the cooresponding coordinates: [-1, 1]
Current maximum proportion 75.76 and the cooresponding coordinates: [-2, 2]
Current maximum proportion 78.34 and the cooresponding coordinates: [-3, 3]
Current maximum proportion 81.10 and the cooresponding coordinates: [-4, 4]
Current maximum proportion 84.11 and the cooresponding coordinates: [-5, 5]
Current maximum proportion 86.59 and the cooresponding coordinates: [-6, 6]
Current maximum proportion 87.78 and the cooresponding coordinates: [-6, 7]
Current maximum proportion 88.79 and the cooresponding coordinates: [-6, 8]
Current maximum proportion 89.50 and the cooresponding coordinates: [-6, 9]
Current maximum proportion 89.84 and the cooresponding coordinates: [-6, 10]
Current maximum proportion 89.92 and the cooresponding coordinates: [-6, 11]


In [18]:
dic_shift_info

[89.91537376586743, [-6, 11]]

# Cellpose part 2

In [19]:
if pretrained_whole_SHIFTCORRECTED_model_path != None:
    shiftcorrected_whole_model = models.CellposeModel(pretrained_model=pretrained_whole_SHIFTCORRECTED_model_path)
    dapi_projection_shifted = shift(dapi_projection,  [-dic_shift_info[1][0],-dic_shift_info[1][1]])
    cellpose_input = [io.imread(corresponding_files[1]), dapi_projection_shifted]
    #
    cellpose_imgs = np.array(cellpose_input)
    nimg = len(imgs)
    #
    shiftmodel_masks_whole, shiftmodel_whole_flows, shiftmodel_whole_styles = shiftcorrected_whole_model.eval(cellpose_imgs, diameter=None, channels=cellpose_channels)

### Per cell mRNA coordinates with the shift 

In [21]:
if pretrained_separate_model_path != None:
    mask_sep_shifted = shift(masks_sep, dic_shift_info[1])

In [22]:
if pretrained_whole_SHIFTCORRECTED_model_path != None:
    mask_whole_shifted = shift(shiftmodel_masks_whole, dic_shift_info[1])
else:
    mask_whole_shifted = shift(masks_whole, dic_shift_info[1])

In [23]:
spots_per_cell_whole_shifted = extracting_spots_per_cell(spot_coord_dict, mask_whole_shifted, projection_dict)
if pretrained_separate_model_path != None:
    spots_per_cell_sep_shifted = extracting_spots_per_cell(spot_coord_dict, mask_sep_shifted, projection_dict)

### Find the corresponding mother and bud segments

In [26]:
# Assign the mother and bud fragments to eachother using the centre points of the masks.
if pretrained_separate_model_path != None:
    centroid_dict = mask_centroids(mapping_mask_coordinates(spots_per_cell_sep_shifted))
    motherbud_pairs = mother_bud_reunion(mapping_mask_coordinates(spots_per_cell_whole_shifted), centroid_dict)
    motherbud_dict = mother_or_bud(motherbud_pairs, spots_per_cell_sep_shifted) # fov index : mother or bud
    print(motherbud_dict)
else:
    motherbud_dict = None

{0: ('bud', 'corresponding mother: 3'), 3: ('mother', 'corresponding bud: 0'), 12: ('mother', 'corresponding bud: 15'), 15: ('bud', 'corresponding mother: 12'), 17: ('bud', 'corresponding mother: 18'), 18: ('mother', 'corresponding bud: 17'), 22: ('bud', 'corresponding mother: 23'), 23: ('mother', 'corresponding bud: 22'), 56: ('bud', 'corresponding mother: 57'), 57: ('mother', 'corresponding bud: 56'), 61: ('bud', 'corresponding mother: 62'), 62: ('mother', 'corresponding bud: 61'), 66: ('mother', 'corresponding bud: 68'), 68: ('bud', 'corresponding mother: 66'), 75: ('mother', 'corresponding bud: 78'), 78: ('bud', 'corresponding mother: 75')}


#### View shifted masks with spots on MIPs with the corresponding DIC

In [27]:
viewer = napari.Viewer()
napari_view(corresponding_files, np.array(projection))
napari_view_spots(corresponding_files, np.array(projection), spot_coord_dict)
napari_view_masks(mapping_mask_coordinates(spots_per_cell_whole_shifted), mask_colors=['pink'])
if pretrained_separate_model_path != None:
    napari_view_masks(mapping_mask_coordinates(spots_per_cell_sep_shifted), mask_colors=['lightblue'])

# Put all the information in a DataFrame

In [28]:
%run -i bigfish_function.py

In [29]:
spot_im_channels = list(projection_dict.keys())
spot_im_channels.remove('DAPIP')

In [40]:
if pretrained_separate_model_path != None:
    sep_df = writing_metrics_to_DataFrame(spots_per_cell_sep_shifted, spot_channels, spot_im_channels, motherbud_dict)
else:
    sep_df = None
whole_df = writing_metrics_to_DataFrame(spots_per_cell_whole_shifted, spot_channels, spot_im_channels, celltype_dictionary=None)

## Save DataFrame as an Excel File

In [41]:
writing_dataframe_to_excel(results_excel_path, whole_df, sep_df)